In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"
model_dir = "model/"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=model_dir)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    cache_dir=model_dir
)

print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [2]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [6]:
prompt = "将一个关于大学生的笑话"
messages = [
    {"role":"system","content":"You are JaLmg, a helpful assistant."},
    {"role":"user","content":prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(text)

<|im_start|>system
You are JaLmg, a helpful assistant.<|im_end|>
<|im_start|>user
将一个关于大学生的笑话<|im_end|>
<|im_start|>assistant



In [7]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(model_inputs)

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,  22871,     43,  12311,     11,
            264,  10950,  17847,     13, 151645,    198, 151644,    872,    198,
          44063,  46944, 101888, 104422,   9370, 109959, 151645,    198, 151644,
          77091,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}


In [8]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs["input_ids"], generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

<think>
好的，用户让我提供一个关于大学生的笑话。首先，我需要确保笑话适合所有年龄段，避免任何可能引起不适的内容。大学生这个群体可能有各种有趣的背景，但要注意幽默的表达方式。

接下来，我得考虑笑话的结构。通常，笑话会有一个问题或情景，然后通过反转或意想不到的方式解决。比如，大学生可能遇到一些常见的问题，但用幽默的方式表达。

然后，我需要检查笑话的连贯性和逻辑性。确保笑话不会让人感到困惑或不适。同时，要保持语言简洁，让读者容易理解。

另外，还要注意笑话的长度，不要太长，适合快速阅读。可能需要调整句子的结构，使其更流畅。

最后，确保笑话符合大学生的常见场景，比如校园生活、学习压力等，这样更容易引起共鸣。同时，加入一些生动的细节，比如表情、动作，可以让笑话更生动有趣。

总结一下，我需要构思一个简单明了、有反转和幽默感的笑话，围绕大学生的生活或问题展开，确保内容健康积极，适合目标读者。
</think>

当然！这是一个关于大学生的幽默笑话：

**"为什么大学生要考研究生？因为……因为……因为……"**

（突然，一个男生从教室里跑出来，手里拿着一个发光的盒子，嘴里喊着："为什么大学生要考研究生？因为……因为……因为……"）

（镜头拉远，一个穿着校服的男生站在教室门口，手里拿着一个发光的盒子，笑得大笑）  

**"因为……因为……因为……"**  

（镜头定格，男生露出开心的笑容，盒子里掉出一张写着"**考研究生**"的纸条。）  

**笑点解析**：  
- 通过反转（盒子里的纸条）制造悬念，  
- 用"为什么"引出荒诞的逻辑，  
- 结尾的"考研究生"点明主题，  
- 用表情和动作增强画面感。


In [9]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=512,
    do_sample=False
)

messages = [
    {"role":"user","content":"请介绍一下你自己"}
]

output = generator(messages)
print(output[0]["generated_text"])

Device set to use cpu
d:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
d:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
d:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


<think>
好的，用户让我介绍一下自己。首先，我需要确定用户的需求是什么。他们可能是在测试我的能力，或者想了解我的背景，或者只是好奇我的身份。作为AI助手，我应该保持自然和友好的语气，同时提供有用的信息。

接下来，我需要考虑用户可能的背景。他们可能是学生、工作人士，或者对AI感兴趣的人。不同的用户可能需要不同的回答。比如，学生可能需要更专业的介绍，而职场人士可能更关注功能和优势。

然后，我需要确保回答的信息准确且全面。要涵盖我的主要功能，比如知识库、对话历史、多语言支持等。同时，要强调我的优点，比如快速响应、灵活调整、隐私保护等，以展示我的可靠性。

还要注意用户可能的深层需求。他们可能希望了解如何使用我的服务，或者如何与我互动。因此，在回答中可以提供一些使用建议，比如如何开始对话，或者如何优化我的回答。

最后，保持回答简洁明了，避免使用过于技术化的术语，让所有用户都能轻松理解。同时，保持礼貌和专业的形象，确保信息传达清晰。
</think>

你好！我是你的AI助手，可以帮你解答各种问题，从学习到工作，从日常事务到创意灵感。如果你有任何问题或需要帮助，请随时告诉我！😊
